In [1]:
import numpy as np
import os
import cv2 as cv

In [3]:
import glob
import cv2 as cv


def getfiles(path):
    print('searching', path)
    files = sorted(glob.glob(path+'/*.png'))

    if not files:
        raise FileNotFoundError(f'no files found under {path}')

    return files


def visualize_image_on_colab(image_path):
    from google.colab.patches import cv2_imshow

    files = getfiles(image_path)

    if not files:
        raise FileNotFoundError(f'No files found under {image_path}')
    
    for fn in files:
        image = cv.imread(fn)
        cv2_imshow(image) # Note cv2_imshow, not cv2.imshow


def visualize_video_on_colab(video_path):
    from google.colab.patches import cv2_imshow

    cap = cv.VideoCapture(video_path)
    while cap.isOpened():
        ret, image = cap.read()

        if not ret:
            break

        cv2_imshow(image) # Note cv2_imshow, not cv2.imshow
        
        cv.waitKey(1) & 0xff

    cv.destroyAllWindows()
    cap.release()

In [6]:
input_dir = "/home/roz/workspace/depth net/dataset/depth_selection/val_selection_cropped/image"
input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png")
    ]
)
print("Number of samples:", len(input_img_paths))

Number of samples: 900


In [11]:
def fb_from_image(path):
    flist = getfiles(path)
    os.makedirs('./fb_results/')

    for i in range(len(flist) - 1):
        im1 = cv.imread(flist[i])
        im2 = cv.imread(flist[i+1])
        gray1 = cv.cvtColor(im1, cv.COLOR_BGR2GRAY)
        gray2 = cv.cvtColor(im2, cv.COLOR_BGR2GRAY)

        hsv = np.zeros_like(im1)
        # Sets image saturation to maximum
        hsv[..., 1] = 255
        # Calculates dense optical flow by Farneback method
        # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
        flow = cv.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Computes the magnitude and angle of the 2D vectors
        magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
        # Sets image hue according to the optical flow direction
        hsv[..., 0] = angle * 180 / np.pi / 2
        # Sets image value according to the optical flow magnitude (normalized)
        hsv[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
        # Converts HSV to RGB (BGR) color representation
        bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

        cv.imwrite(f'./fb_results/optical_fb_{i+1}.png',bgr)

In [13]:
fb_from_image("/home/roz/workspace/depth net/dataset/depth_selection/val_selection_cropped/image")

searching /home/roz/workspace/depth net/dataset/depth_selection/val_selection_cropped/image


In [18]:
imm1 = cv.imread("/home/roz/workspace/depth net/dataset/depth_selection/val_selection_cropped/image/2011_09_26_drive_0002_sync_image_0000000005_image_02.png")
imm2 = cv.imread("/home/roz/workspace/depth net/dataset/depth_selection/val_selection_cropped/image/2011_10_03_drive_0047_sync_image_0000000488_image_03.png")
grayy1 = cv.cvtColor(imm1, cv.COLOR_BGR2GRAY)
grayy2 = cv.cvtColor(imm2, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(imm1)
        # Sets image saturation to maximum
hsv[..., 1] = 255
        # Calculates dense optical flow by Farneback method
        # https://docs.opencv.org/3.0-beta/modules/video/doc/motion_analysis_and_object_tracking.html#calcopticalflowfarneback
flow = cv.calcOpticalFlowFarneback(grayy1, grayy2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Computes the magnitude and angle of the 2D vectors
magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
        # Sets image hue according to the optical flow direction
hsv[..., 0] = angle * 180 / np.pi / 2
        # Sets image value according to the optical flow magnitude (normalized)
hsv[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
        # Converts HSV to RGB (BGR) color representation
bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
cv.imwrite(f'./fb_results/optical_fb_900.png',bgr)

True